Hello World!

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import attila_utils
from bsmcalls import SNPnexus
from bsmcalls import operations
from bsmcalls import resources
from bsmcalls import individuals
from bsmcalls import stats as bsmstats
import statsmodels.api as sm
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels import graphics as smg
from statsmodels.graphics import regressionplots
import patsy
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


FileNotFoundError: [Errno 2] No such file or directory: '/home/attila/projects/bsm/resources/cmc-ancestry/CMC_MSSM-Penn-Pitt_DNA_GENOTYPE_ANCESTRY_GemTools.tsv'

In [ ]:
data = SNPnexus.load_data('/home/attila/projects/bsm/results/2020-09-07-annotations/annotated-calls.p')
clozukpath = '/home/attila/projects/bsm/resources/CLOZUK/supp-table-4.csv'
gwasgenes = operations.get_geneset(df=pd.read_csv(clozukpath, skiprows=7), col='Gene(s) tagged')

In [ ]:
# near_gens
querydict = {'near_gens_Annotation': ['coding nonsyn', 'coding syn', 'missense', 'stop-gain', 'intronic (splice_site)'],
             'ensembl_Predicted Function': ['coding'],
             'sift_Prediction': ['Deleterious', 'Deleterious - Low Confidence'],
             'polyphen_Prediction': ['Probably Damaging', 'Possibly Damaging'],
             'tfbs_TFBS Name': None,
             'phast_Score': None,
             'gerp_Element RS Score': None,
             'cpg_CpG Island': None,
             'near_gens_Overlapped Gene': {'SCZ GWAS genes': gwasgenes},
            }
results = operations.multiquery(querydict, data, do_sum=False, do_sort=False)
summary = operations.summarize_query_results(results, data, chisq=False, margin=False)
summary#.style.bar(subset='chisq stat')

In [ ]:
d = {'ncalls': data.groupby('Dx').size(), 'nsamples': individuals.get_nsamples(results)}
dfd = {k: operations.chisquare_summary(summary, expected_odds, append=False) for k, expected_odds in d.items()}
dfl = [pd.DataFrame(df.to_numpy(), columns=pd.MultiIndex.from_product([[k], df.columns]), index=df.index) for k, df in dfd.items()]
df = pd.concat(dfl, axis=1)
df

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1, 2, sharey=True)
d['ncalls'].plot(kind='barh', ax=ax[0])
pd.Series(d['nsamples']).plot(kind='barh', ax=ax[1])
ax[0].set_title('Genomewide calls')
ax[1].set_title('Individuals')

## Limitations of the $\chi^2$ test

* univariate: it doesn't allow easy analysis of joint effects of multiple features
* counts of calls are the only possible variable to model

### Example: calls weighted by allele frequency

This piece of analysis shows that weighting each call with its allele frequency decreases significance of the $\chi^2$ test since the $\mathrm{AF} < 1$ (moreover $\mathrm{AF} \ll 1$ for most calls).

In [ ]:
wncalls = data.groupby('Dx')['AF'].sum()
wresults = results.drop(('Dx', ), axis=1).astype('int16').apply(lambda x: x * data['AF'], axis=0)
operations.chisquare_summary(summary, expected_odds=wncalls, append=False)

## DataFrame for regression: `fitdata`

In [ ]:
selcols = ['DP', 'BaseQRankSum', 'AF']
covariates = data[selcols].groupby('Individual ID').mean()
selcols_indiv = ['Dx', 'ageOfDeath', 'Dataset']
covariates[selcols_indiv] = data[selcols_indiv].groupby('Individual ID').first()
covariates['ncalls'] = data.groupby('Individual ID').size()
covariates = pd.concat([covariates, covariates['ncalls'].apply(np.log10).rename('log10_ncalls')], axis=1)
covariates

In [ ]:
Dxs = pd.Categorical(results[('Dx', )].groupby('Individual ID').first(), categories=results[('Dx', )].cat.categories)
responses = results.drop(('Dx',), axis=1).groupby('Individual ID').sum()
prettynames = ['coding_nonsyn', 'coding_syn', 'missense', 'stop_gain', 'splice_site', 'coding', 'deleterious', 'deleterious_low_confidence', 'probably_damaging', 'possibly_damaging', 'tfbs', 'phast', 'gerp', 'cpg_island', 'scz_gwas_genes']
responses.columns = prettynames
responses.info()

In [ ]:
responses_prop = responses.apply(lambda x: x / covariates['ncalls'], axis=0).rename(lambda x: 'prop_' + x, axis=1)

In [ ]:
fitdata = pd.concat([responses, covariates], axis=1)

df = pd.DataFrame(data.groupby('Individual ID').size(), columns=['ncalls'])
df['ncalls_scz_gwas_genes'] = results[('near_gens_Overlapped Gene', 'SCZ GWAS genes')].groupby('Individual ID').sum()
df['fcalls_scz_gwas_genes'] = df['ncalls_scz_gwas_genes'] / df['ncalls']
selindivcols = ['Dx', 'ageOfDeath']
df[selindivcols] = data[selindivcols].groupby('Individual ID').first()
selcols = ['DP', 'BaseQRankSum', 'AF']
df[selcols] = data[selcols].groupby('Individual ID').mean()
scz_gwas_genes = df
scz_gwas_genes

### Pairwise joint distributions

* No two variables show extremely tight dependence which implies limited collinearity in a normal or generalized linear model.
* The fraction of calls `fcalls_scz_gwas_genes` has very high variance when the total number of calls for an individual is low. This is undesirable.

In [ ]:
df = pd.concat([responses['scz_gwas_genes'], responses_prop['prop_scz_gwas_genes'], covariates], axis=1)
ax = pd.plotting.scatter_matrix(df, figsize=(15, 15))

In [ ]:
fig, ax = plt.subplots()
for Dx in ['Control', 'SCZ', 'ASD']:
    ax.scatter(x='ncalls', y='scz_gwas_genes', data=fitdata.loc[fitdata['Dx'] == Dx], label=Dx)
ax.legend()

## Multiple features/functional categories

In [ ]:
Dxs = fitdata['Dx'].copy()
Dxcol = Dxs.cat.rename_categories(['C0', 'C1', 'C2'])

def foo(responses, covariates, Dxcol, dropASD=True):
    covariates = covariates.select_dtypes(exclude='category')
    if dropASD:
        responses = responses.loc[Dxcol != 'C2']
        covariates = covariates.loc[Dxcol != 'C2']
        Dxcol = Dxcol[Dxcol != 'C2']

    nresp = responses.shape[1]
    ncovar = covariates.shape[1]
    fig, ax = plt.subplots(nresp, ncovar, sharey=False, figsize=(ncovar * 2, nresp * 2))
    for i, row in zip(range(nresp), responses.columns):
        response = responses[row]
        ax[i, 0].set_ylabel(row, rotation='horizontal', horizontalalignment='right')
        for j, col in zip(range(ncovar), covariates.columns):
            if i == 0:
                ax[i, j].set_title(col)
            if i == nresp - 1:
                ax[i, j].set_xlabel(col)
            ax[i, j].scatter(y=response, x=covariates[col], marker='|', c=Dxcol)

foo(responses, covariates, Dxcol, False)

In [ ]:
foo(responses_prop, covariates, Dxcol, False)

### More transformations

In [ ]:
%matplotlib inline
features = ['DP', 'AF']
fig, ax = plt.subplots(len(features), 2, figsize=(10, 10), sharey=True)
for i, feature in zip(range(len(features)), features):
    ax[i, 0].scatter(y=responses['scz_gwas_genes'], x=covariates[feature], marker='|')
    ax[i, 0].set_ylabel('scz_gwas_genes')
    ax[i, 0].set_xlabel(feature)
    ax[i, 1].scatter(y=responses['scz_gwas_genes'], x=np.log10(covariates[feature]), marker='|')
    ax[i, 1].set_xlabel('$\log_{10}$ ' + feature)

## Fitting models

### Training data
First simplify column names and create data matrix

In [ ]:
fitdata = pd.concat([responses, covariates], axis=1)
fitdata['Dx'] = Dxs
endog = responses
exog0 = fitdata.drop(prettynames + ['ncalls'], axis=1)
exog1 = fitdata.drop(prettynames + ['log10_ncalls'], axis=1)

In [ ]:
def endog_binomial(feature, fitdata=fitdata, proportion=False):
    success = fitdata[feature]
    if proportion:
        prop = success / fitdata['ncalls']
        return(prop)
    failure = fitdata['ncalls'] - success
    complement = 'NOT_' + feature
    df = pd.DataFrame({feature: success, complement: failure})
    return(df)

### Log linear (Poisson) models

In [ ]:
formula = 'scz_gwas_genes ~ Dx'
y0, X0 = patsy.dmatrices(formula, data=fitdata, return_type='dataframe')
pois0 = sm.GLM(endog=y0, exog=X0, family=sm.families.Poisson()).fit()

In [ ]:
formula = 'scz_gwas_genes ~ Dx + log10_ncalls'
y1, X1 = patsy.dmatrices(formula, data=fitdata, return_type='dataframe')
pois1 = sm.GLM(endog=y1, exog=X1, family=sm.families.Poisson()).fit()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(6, 6))
left = fitdata['log10_ncalls'].min()
right = fitdata['log10_ncalls'].max()
log10_ncalls_test = np.linspace(left, right)
Dxd = {'Control': (0, 0), 'SCZ': (1, 0), 'ASD': (0, 1)}
for color, Dx in zip(['C0', 'C1', 'C2'], fitdata['Dx'].cat.categories):
    y0_pred = pois0.predict([1, *Dxd[Dx]])
    ax.plot([left, right], [y0_pred] * 2, linestyle='solid', label=Dx + '0')
    X_test = pd.DataFrame(dict(zip(X1.columns, [1, *Dxd[Dx], log10_ncalls_test])))
    ax.scatter(x='log10_ncalls', y='scz_gwas_genes', data=fitdata.loc[fitdata['Dx'] == Dx], marker='|', color=color)
    ax.plot(log10_ncalls_test, pois1.predict(X_test), color=color, label=Dx)
ax.legend(Dxd.keys())

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
g = pois1.plot_partial_residuals('log10_ncalls', ax=ax[0])
g = pois1.plot_partial_residuals('log10_ncalls', ax=ax[1])
ax[1].set_ylim(0, 10)

### Poisson model results

In [ ]:
formula = 'scz_gwas_genes ~ Dx + log10_ncalls + DP'
y2, X2 = patsy.dmatrices(formula, data=fitdata, return_type='dataframe')
pois2 = sm.GLM(endog=y2, exog=X2, family=sm.families.Poisson()).fit()

In [ ]:
formula = 'scz_gwas_genes ~ Dx + log10_ncalls + np.log(DP)'
y2b, X2b = patsy.dmatrices(formula, data=fitdata, return_type='dataframe')
pois2b = sm.GLM(endog=y2b, exog=X2b, family=sm.families.Poisson()).fit()

The residual deviance of the two models are very close to each other

In [ ]:
pd.DataFrame({'pois2': pois2.deviance, 'pois2b': pois2b.deviance}, index=['deviance'])

The p-values of the two models are very close to each other

In [ ]:
pd.DataFrame({'pois2': pois2.pvalues, 'pois2b': pois2b.pvalues})

In [ ]:
formula = 'scz_gwas_genes ~ Dx + log10_ncalls + DP + AF + ageOfDeath + Dataset'
y, X = patsy.dmatrices(formula, data=fitdata, return_type='dataframe')
pois = sm.GLM(endog=y, exog=X, family=sm.families.Poisson()).fit()
pois.summary()

### Logistic regression (binomial error model)

#### Full model

In [ ]:
formula = 'Dx + DP + AF + ageOfDeath + Dataset'
X = patsy.dmatrix(formula, data=fitdata, return_type='dataframe')

Response can be given as either:
1. counts of both success and failure
1. proportions

In [ ]:
success_failure = endog_binomial('scz_gwas_genes', fitdata, False)
proportion = endog_binomial('scz_gwas_genes', fitdata, proportion=True)
pd.concat([success_failure, proportion, fitdata['ncalls']], axis=1).rename({0: 'proportion'}, axis=1)

When using proportion type response then the total number of calls `fitdata['ncalls']` must be passed as `var_weights` to `GLM`.  Note that the two fits result in the same parameter estimates.

In [ ]:
# Method: response is a success failure pair
mod = sm.GLM(endog=success_failure, exog=X, family=sm.families.Binomial()).fit()
# Method 2: response is a proportion; var_weights must be specified
mod_p = sm.GLM(endog=proportion, exog=X, family=sm.families.Binomial(), var_weights=fitdata['ncalls']).fit()
# Check if parameter estimates are identical
all(mod.params == mod_p.params)

In [ ]:
mod.summary()

#### Intermediate model

In [ ]:
formula = 'Dx + ageOfDeath + Dataset'
X1 = patsy.dmatrix(formula, data=fitdata, return_type='dataframe')
mod1 = sm.GLM(endog=success_failure, exog=X1, family=sm.families.Binomial()).fit()
mod1.summary()

#### Minimal model

In [ ]:
formula = 'Dx + ageOfDeath'
X0 = patsy.dmatrix(formula, data=fitdata, return_type='dataframe')
mod0 = sm.GLM(endog=success_failure, exog=X0, family=sm.families.Binomial()).fit()
mod0.summary()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ageOfDeath_test = np.arange(0, 120)
#X_test = pd.DataFrame(dict(zip(X0.columns, [1, 0, 0, ageOfDeath_test])))
for color, Dx in zip(['C0', 'C1', 'C2'], fitdata['Dx'].cat.categories):
    X_test = pd.DataFrame(dict(zip(X0.columns, [1, *Dxd[Dx], ageOfDeath_test])))
    def myscatter(ax):
        return(ax.scatter(y=proportion.loc[fitdata['Dx'] == Dx], x=fitdata.loc[fitdata['Dx'] == Dx, 'ageOfDeath'], marker='|', color=color))
    def myplot(ax):
        return(ax.plot(ageOfDeath_test, mod0.predict(X_test), color=color, label=Dx))
    myscatter(ax[0])
    myplot(ax[0])
    myscatter(ax[1])
    myplot(ax[1])
    ax[0].set_ylabel('ncalls in scz_gwas_genes / ncalls')
    ax[0].set_xlabel('ageOfDeath')
    ax[0].legend()
    ax[1].set_ylabel('ncalls in scz_gwas_genes / ncalls')
    ax[1].set_xlabel('ageOfDeath')
    ax[1].legend()
    ax[1].set_ylim([-0.01, 0.10])

## Other features

In [ ]:
formula = 'Dx + DP + AF + ageOfDeath + Dataset'
X = patsy.dmatrix(formula, data=fitdata, return_type='dataframe')
endognames = ['coding_nonsyn', 'coding_syn', 'coding', 'deleterious', 'deleterious_low_confidence', 'probably_damaging', 'possibly_damaging', 'tfbs', 'phast', 'gerp', 'cpg_island', 'scz_gwas_genes']
proportions = {endog: endog_binomial(endog, fitdata, proportion=True) for endog in endognames}
mods = {endog: sm.GLM(endog=proportions[endog], exog=X, family=sm.families.Binomial(), var_weights=fitdata['ncalls']).fit() for endog in proportions.keys()}

In [ ]:
%matplotlib inline
def my_dotplot(feature):
    tvalues = pd.Series({endog: mods[endog].tvalues[feature] for endog in mods.keys()})
    pvalues = pd.Series({endog: mods[endog].pvalues[feature] for endog in mods.keys()})
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(feature + ' significance')
    ax[0].plot([0, 0], [0, 12])
    g = smg.dotplots.dot_plot(tvalues, lines=tvalues.index, ax=ax[0])
    ax[0].set_xlim(np.array([-1, 1]) * 1.1 * tvalues.abs().max())
    ax[0].set_xlabel('t-value')
    g = smg.dotplots.dot_plot(pvalues, lines=pvalues.index, ax=ax[1], show_names='right')
    ax[1].set_xscale('log')
    ax[1].set_xlabel('p-value')
    return((fig, ax))

fig, ax = my_dotplot('Dx[T.SCZ]')

In [ ]:
fig, ax = my_dotplot('Dx[T.ASD]')

In [ ]:
fig, ax = my_dotplot('ageOfDeath')

In [ ]:
%connect_info